In [143]:
import requests

from bs4 import BeautifulSoup

import smtplib

from email.message import EmailMessage

from datetime import datetime, timedelta, date

In [144]:
yesterday = date.today() - timedelta(days=1)

brentwood_rei_url = 'https://rei.jobs/careers/SearchJobs/?3_73_3=37158&jobSort=postedDate&jobSortDirection=DESC&'
lebanon_rei_url = 'https://rei.jobs/careers/SearchJobs/?3_73_3=5813561&jobSort=postedDate&jobSortDirection=DESC&'
chattanooga_rei_url = 'https://rei.jobs/careers/SearchJobs/?3_73_3=2217122&jobSort=postedDate&jobSortDirection=DESC&'

In [145]:
def get_job_titles(url, dict_to_append):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    joblist = soup.find('ul', class_='jobList')
    for li in joblist.find_all('li', class_="group"):
        title = li.find('a').get('title')
        link = li.find('a').get('href')
        if li.find('p') is None:
            pass
        else:
            date_str = li.find('p').find_all('span')[1].text.split(' ', 1)[1]
            date_obj = datetime.strptime(date_str, '%m-%d-%Y').date()
             
        dict_to_append.update({link:[title, date_obj]})

In [146]:
job_dict = {}

get_job_titles(brentwood_rei_url, job_dict)
get_job_titles(lebanon_rei_url, job_dict)
get_job_titles(chattanooga_rei_url, job_dict)

In [147]:
new_jobs_str = ''

for (k,v) in job_dict.items():
    if v[1] > yesterday:
        new_jobs_str += f'{str(v[0])}\n{k}\n\n'
    else:
        pass

In [148]:
if len(new_jobs_str) == 0:
    print('No new jobs')
else:
    smtpHost = 'smtp.office365.com'
    smtpPort = 587
    sender = 'georgetregoning@gmail.com'
    with open('outlookpassword.txt') as f:
        password = f.readlines()[0]

    receiver = ['georgetregoning@gmail.com']
    
    subject = "New Brentwood-Chatt-Lebanon REI Jobs"
    # Add the From: and To: headers at the start!
    message = ("From: %s\r\nTo: %s\r\nSubject: %s\r\n\r\n"
       % (sender, ", ".join(receiver), subject))
    message += new_jobs_str
    
    try:
        smtpObj = smtplib.SMTP(smtpHost, smtpPort)
        #smtpObj.set_debuglevel(1)
        smtpObj.ehlo()
        smtpObj.starttls()
        smtpObj.ehlo()    
        smtpObj.login(sender,password)
        smtpObj.sendmail(sender, receiver, message)
        smtpObj.quit()
        print ("Successfully sent email")
    except:
        print ("Error: unable to send email")

Successfully sent email
